In [33]:
import pandas as pd 
import deepchem as dc

df = pd.read_csv('need_fill_nullvalue.csv')
df.head()

,canon_smiles,herg_pIC50,hepg2_pIC50,cyp2c9_pIC50,cyp2d6_pIC50,cyp3a4_pIC50
0,Br.CCCCOc1ccc2c3ccnc(C)c3n(CC(C)C)c2c1,NaN,-4.081393,NaN,NaN,NaN
1,Br.CCCCOc1ccc2c3ccnc(C)c3n(CCCC)c2c1,NaN,-4.190166,NaN,NaN,NaN
2,Br.CCCCn1c2cc(O)ccc2c2ccnc(C)c21,NaN,-4.910579,NaN,NaN,NaN
3,Br.CCCCn1c2cc(OC(C)C)ccc2c2ccnc(C)c21,NaN,-4.480003,NaN,NaN,NaN
4,Br.CCCCn1c2cc(OCC)ccc2c2ccnc(C)c21,NaN,-4.638244,NaN,NaN,NaN


In [ ]:
import os
import tempfile
tempdir = tempfile.mkdtemp()

smiles = df[~df['herg_pIC50'].isnull()]['canon_smiles']
labels = df[~df['herg_pIC50'].isnull()]['herg_pIC50']

df = pd.DataFrame(list(zip(smiles, labels)), columns=["smiles", "task1"])
with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:
    df.to_csv(tmpfile.name)
    loader = dc.data.CSVLoader(["task1"], feature_field="smiles", featurizer=dc.feat.DummyFeaturizer())
    dataset = loader.create_dataset(tmpfile.name)
    
from deepchem.models.torch_models.chemberta import Chemberta
pretrain_model_dir = os.path.join(tempdir, 'pretrain-model')
tokenizer_path = "seyonec/PubChem10M_SMILES_BPE_60k"
pretrain_model = Chemberta(task='mlm', model_dir=pretrain_model_dir, tokenizer_path=tokenizer_path)  # mlm pretraining
pretraining_loss = pretrain_model.fit(dataset, nb_epoch=1)

finetune_model_dir = os.path.join(tempdir, 'finetune-model')
finetune_model = Chemberta(task='regression', model_dir=finetune_model_dir, tokenizer_path=tokenizer_path)
finetune_model.load_from_pretrained(pretrain_model_dir)
finetuning_loss = finetune_model.fit(dataset, nb_epoch=1)

result = finetune_model.predict(dataset)
eval_results = finetune_model.evaluate(dataset, metrics=dc.metrics.Metric(dc.metrics.mae_score))